In [163]:
import pandas as pd
import datetime
import time
import math

In [186]:
pd.set_option('float_format', '{:f}'.format)
folder = '24_12_2018_15_54_TH_r'
res_adapt = pd.read_csv(folder + '/adapt.csv')
res_adapt_eval = pd.read_csv(folder + '/adapt_eval.csv')
res_eval = pd.read_csv(folder + '/eval.csv')

In [187]:
res_adapt.drop(res_adapt[res_adapt.instance.str.contains('exact')].index, inplace=True)
res_eval.drop(res_eval[res_eval.instance.str.contains('exact')].index, inplace=True)
res_adapt_eval.drop(res_adapt_eval[res_adapt_eval.instance.str.contains('exact')].index, inplace=True)

In [188]:
res_corr = res_eval.merge(res_adapt, left_on='instance', right_on='instance', how='inner')

In [189]:
res_corr = res_corr.fillna(0)
res_corr['diff_p'] = (res_corr['pred_p'] - res_corr['real_p'])
res_corr['diff_p'] = res_corr['diff_p']*res_corr['diff_p']

In [190]:
df1 = res_corr[res_corr['type_x'] == 'FeatureBased_SVM']['pred_p'].reset_index()
df2 = res_corr[res_corr['type_x'] == 'CRNN']['cos'].reset_index()
pd.concat([df1, df2], axis=1).corr().loc['pred_p', 'cos']

0.015253169314978695

In [191]:
res_corr.groupby('type_x')[['pred_p', 'old_p', 'old_r', 'old_f', 'cos', 'sum']].corr()

pred_p     old_p  \
type_x                                                                  
CRNN                                       pred_p  1.000000  0.152057   
                                           old_p   0.152057  1.000000   
                                           old_r  -0.157748  0.689427   
                                           old_f   0.241477  0.963780   
                                           cos     0.008752  0.007417   
                                           sum    -0.446134 -0.599094   
DNN                                        pred_p  1.000000  0.002059   
                                           old_p   0.002059  1.000000   
                                           old_r  -0.277908  0.689427   
                                           old_f   0.041392  0.963780   
                                           cos     0.015787  0.007417   
                                           sum    -0.439169 -0.599094   
FeatureBased_Bayesian Ridge Regression     pred_p  1.000000 -0.402484   
                                           old_p  -0.402484  1.000000   
                                           old_r  -0.473717  0.689427   
                                           old_f  -0.321201  0.963780   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_Lasso Regression              pred_p  1.000000 -0.402484   
                                           old_p  -0.402484  1.000000   
                                           old_r  -0.473717  0.689427   
                                           old_f  -0.321201  0.963780   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_Linear Regression             pred_p  1.000000 -0.402484   
                                           old_p  -0.402484  1.000000   
                                           old_r  -0.473717  0.689427   
                                           old_f  -0.321201  0.963780   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_Passive Aggressive Regression pred_p  1.000000 -0.076103   
                                           old_p  -0.076103  1.000000   
                                           old_r  -0.023469  0.689427   
                                           old_f  -0.127787  0.963780   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_SGD Regression                pred_p  1.000000 -0.141658   
                                           old_p  -0.141658  1.000000   
                                           old_r  -0.103022  0.689427   
                                           old_f  -0.173720  0.963780   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_SVM                           pred_p  1.000000 -0.402484   
                                           old_p  -0.402484  1.000000   
                                           old_r  -0.473717  0.689427   
                                           old_f  -0.321201  0.963780   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_Theil-Sen Regression          pred_p  1.000000 -0.064788   
                                           old_p  -0.064788  1.000000   
                                           old_r  -0.010019  0.689427   
                                           old_f  -0.119555  0.963780   
                                           cos          nan       nan   
                                      

In [192]:
res_adapt['diff_p'] = res_adapt['new_p'] - res_adapt['old_p']
res_adapt['diff_r'] = res_adapt['new_r'] - res_adapt['old_r']
res_adapt['diff_f'] = res_adapt['new_f'] - res_adapt['old_f']

In [193]:
res_adapt['pair'], res_adapt['alg'] = res_adapt['instance'].str.split(' ', 1).str
res_adapt2 = res_adapt[~(res_adapt['alg'].isin(['AMC Token Path', 'WordNet Jiang Conrath', 'Ontobuilder Term Match']))]

In [194]:
res_adapt.groupby('type').mean()

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,
BPR,917,0,0.447726,0.553540,0.451664,0.038065,0.085662,0.017440,-0.409660,-0.467878,-0.434224
CRNN,919,0,0.447726,0.553540,0.451664,0.467977,0.526277,0.479494,0.020251,-0.027263,0.027830
DNN,918,0,0.447726,0.553540,0.451664,0.528948,0.568981,0.534994,0.081222,0.015440,0.083330
IR,915,0,0.447726,0.553540,0.451664,0.504519,0.529412,0.516562,0.056794,-0.024128,0.064898
MULTI,920,0,0.447726,0.553540,0.451664,0.501514,0.524626,0.511787,0.053789,-0.028914,0.060123
SVDpp,916,0,0.447726,0.553540,0.451664,0.484665,0.525493,0.492847,0.036940,-0.028047,0.041183


In [196]:
res_adapt2.groupby('type').mean()

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,
BPR,903.694118,0.000000,0.509685,0.568698,0.522251,0.030305,0.034880,0.011971,-0.479380,-0.533818,-0.510280
CRNN,905.694118,0.000000,0.509685,0.568698,0.522251,0.489197,0.529572,0.500045,-0.020488,-0.039126,-0.022207
DNN,904.694118,0.000000,0.509685,0.568698,0.522251,0.539077,0.569097,0.544234,0.029392,0.000399,0.021983
IR,901.694118,0.000000,0.509685,0.568698,0.522251,0.504519,0.529412,0.516562,-0.005165,-0.039287,-0.005690
MULTI,906.694118,0.000000,0.509685,0.568698,0.522251,0.500913,0.523669,0.510832,-0.008772,-0.045030,-0.011419
SVDpp,902.694118,0.000000,0.509685,0.568698,0.522251,0.487827,0.526228,0.496349,-0.021858,-0.042471,-0.025902


In [135]:
res_adapt_eval['final_p'] = res_adapt_eval[['old_p', 'new_p']].max(axis=1)
res_adapt_eval['final_r'] = res_adapt_eval[['old_r', 'new_r']].max(axis=1)
res_adapt_eval['final_f'] = res_adapt_eval[['old_f', 'new_f']].max(axis=1)

In [136]:
# idx = res_adapt_eval.groupby([['instance', 'type']])['k'].transform(max) == df['k']
# best_k = res_adapt_eval[idx]
res_adapt_eval_k = res_adapt_eval.sort_values('final_f', ascending=False).drop_duplicates(['instance', 'type'])

In [137]:
res_adapt_eval_k.groupby('type').mean()[['final_p', 'final_r', 'final_f']]

,final_p,final_r,final_f
type,,,
CRNN_CRNN,0.503712,0.568199,0.515978
MULTI,0.540887,0.572112,0.551908


In [138]:
res_adapt_eval_k['pair'], res_adapt_eval_k['alg'] = res_adapt_eval_k['instance'].str.split(' ', 1).str
res_adapt_eval_max = res_adapt_eval_k.sort_values('pred_e', ascending=False).drop_duplicates(['pair'])
res_adapt_eval_max.groupby('type').mean()[['final_p', 'final_r', 'final_f']]

,final_p,final_r,final_f
type,,,
CRNN_CRNN,0.469805,0.500000,0.484404
MULTI,0.518983,0.541667,0.529961


In [105]:
res_adapt_eval[res_adapt_eval['type'] =='MULTI'].groupby('pair').max()[['final_p', 'final_r', 'final_f']].mean()

final_p   0.779419
final_r   0.826513
final_f   0.774038
dtype: float64

In [102]:
res_adapt['new_f_max'] = res_adapt.groupby(['pair', 'type'])['new_f'].transform(max)
clean_a = res_adapt[res_adapt['new_f_max'] == res_adapt['new_f']]
clean_a.groupby('type').mean()
# res_adapt['diff_f_max'] = res_adapt.groupby(['pair'])['diff_f'].transform(max)
# clean_a = res_adapt[res_adapt['diff_f_max'] == res_adapt['diff_f']]
# clean_a.groupby('type').mean()
# clean_a.groupby('type').mean()
# grouped = clean_a[clean_a['type'] == 'CNN'].groupby(['type', 'pair']).mean().reset_index()
# grouped.sort_values('diff_f', ascending=False)

KeyError: 'pair'

In [133]:
CRNN_adapt = res_adapt[res_adapt['type'] =='CRNN']

In [134]:
CRNN_adapt['new_f_max'] = CRNN_adapt.groupby(['pair'])['new_f'].transform(max)

C:\Users\shrag\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [135]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [137]:
CRNN_adapt_max.mean()

Unnamed: 0   3964.323232
k               0.000000
old_p           0.651197
old_r           0.750034
old_f           0.646826
new_p           0.733820
new_r           0.713100
new_f           0.705204
diff_p          0.082623
diff_r         -0.036934
diff_f          0.058378
pair                 inf
new_f_max       0.705204
dtype: float64

In [138]:
res_adapt_eval['pred_e_max'] = res_adapt_eval.groupby(['instance','type'])['real_e'].transform(max)
res_adapt_eval['k_max'] = res_adapt_eval.groupby(['instance','type'])['k'].transform(max)
clean_a_e = res_adapt_eval[(res_adapt_eval['pred_e_max'] == res_adapt_eval['real_e']) &
               (res_adapt_eval['k_max'] == res_adapt_eval['k'])]

In [139]:
res_adapt_eval

,Unnamed: 0,instance,type,k,old_p,old_r,old_f,new_p,new_r,new_f,real_e,pred_e,pred_e_max,k_max
0,0,1 AMC Token Path Ontobuilder Max Delta,GRU_GRU,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.463253,0.458160,0.463253,1
1,1,1 AMC Token Path Ontobuilder Max Delta,GRU_GRU,1,0.512335,0.768540,0.473108,0.512335,0.768540,0.473108,0.463253,0.680636,0.463253,1
2,2,1 AMC Token Path Ontobuilder Max Delta,CNN_CNN,0,0.510861,0.756943,0.463253,0.496379,0.499669,0.498018,0.463253,0.899688,0.463253,1
3,3,1 AMC Token Path Ontobuilder Max Delta,CNN_CNN,1,0.496379,0.499669,0.498018,0.496379,0.499669,0.498018,0.463253,0.967738,0.463253,1
4,4,1 AMC Token Path Ontobuilder Max Delta,DNN_DNN,0,0.510861,0.756943,0.463253,0.496382,0.500000,0.498184,0.463253,0.674669,0.463253,0
5,5,1 AMC Token Path Ontobuilder Max Delta,CRNN_CRNN,0,0.510861,0.756943,0.463253,0.496360,0.497018,0.496689,0.463253,0.300762,0.463253,1
6,6,1 AMC Token Path Ontobuilder Max Delta,CRNN_CRNN,1,0.496360,0.497018,0.496689,0.496360,0.497018,0.496689,0.463253,0.814879,0.463253,1
7,7,1 AMC Token Path Ontobuilder Max Delta,GRU_CNN,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.463253,0.899688,0.463253,0
8,8,1 AMC Token Path Ontobuilder Max Delta,GRU_DNN,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.463253,0.674669,0.463253,1
9,9,1 AMC Token Path Ontobuilder Max Delta,GRU_DNN,1,0.512335,0.768540,0.473108,0.512335,0.768540,0.473108,0.463253,0.704076,0.463253,1


In [140]:
clean_a_e.groupby('type').mean()[['new_p', 'new_r', 'new_f']]

,new_p,new_r,new_f
type,,,
BPR_FEATBayesian Ridge Regression,0.510357,0.572868,0.489646
BPR_FEATLasso Regression,0.510357,0.572868,0.489646
BPR_FEATLinear Regression,0.510357,0.572868,0.489646
BPR_FEATPassive Aggressive Regression,0.510357,0.572868,0.489646
BPR_FEATSGD Regression,0.510357,0.572868,0.489646
BPR_FEATSVM,0.510357,0.572868,0.489646
BPR_FEATTheil-Sen Regression,0.510357,0.572868,0.489646
BPR_GRU,0.510357,0.572868,0.489646
CNN_CNN,0.544619,0.554299,0.535674


In [141]:
res_adapt_eval['pair'], res_adapt_eval['alg'] = res_adapt_eval['instance'].str.split(' ', 1).str
CRNN_adapt_eval = res_adapt_eval[res_adapt_eval['type'] =='CRNN']

In [20]:
res_adapt['pair'], res_adapt['alg'] = res_adapt['instance'].str.split(' ', 1).str

In [142]:
CRNN_adapt_eval['new_f_max'] = CRNN_adapt_eval.groupby(['pair'])['new_f'].transform(max)

In [143]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [146]:
CRNN_adapt_max.mean()

Unnamed: 0   3964.323232
k               0.000000
old_p           0.651197
old_r           0.750034
old_f           0.646826
new_p           0.733820
new_r           0.713100
new_f           0.705204
diff_p          0.082623
diff_r         -0.036934
diff_f          0.058378
pair                 inf
new_f_max       0.705204
dtype: float64

In [145]:
def summerize(data, file):
    idMax = data[data['type'] == data['type']]
    if 'pred_e' in data:
        idMax = data.groupby(['type'])['pred_e'].transform(max) == data['pred_e']
        pd.DataFrame(data[idMax].groupby(by='type').mean()).to_csv(file)
    else:
        pd.DataFrame(data.groupby(by='type').mean()).to_csv(file)

In [23]:
data = pd.read_csv('adapt.csv', low_memory=False)
summerize(data, 'a.csv')

FileNotFoundError: File b'adapt.csv' does not exist